# Entrenamiento de Behavioral Cloning CNN
Este notebook entrena una red neuronal convolucional para predecir ángulos de dirección a partir de imágenes capturadas en Webots.

In [ ]:
# Clonar el repositorio con las imágenes y CSV
!git clone https://github.com/juliomestas/navegacion_autonoma.git

In [ ]:
# Librerías necesarias
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.utils import img_to_array
from sklearn.model_selection import train_test_split

In [ ]:
# Cargar imágenes y ángulos
def load_data(base_path):
    df = pd.read_csv(os.path.join(base_path, 'angles.csv'))
    X, y = [], []
    for _, row in df.iterrows():
        img_path = os.path.join(base_path, 'captured_images', row['filename'])
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (200, 66))
            X.append(img_to_array(img))
            y.append(row['vision_angle'])
    return np.array(X), np.array(y)

# Cargar ambos recorridos y combinarlos
X1, y1 = load_data('navegacion_autonoma/controllers/my_vehicle_controller/circuito_ida_con_trapecio')
X2, y2 = load_data('navegacion_autonoma/controllers/my_vehicle_controller/circuito_regreso_con_trapecio')

X = np.concatenate([X1, X2]) / 255.0
y = np.concatenate([y1, y2])

In [ ]:
# Data augmentation (flip horizontal)
X_flipped = np.array([np.fliplr(img) for img in X])
y_flipped = -y
X = np.concatenate([X, X_flipped])
y = np.concatenate([y, y_flipped])

In [ ]:
# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Modelo CNN estilo NVIDIA
model = Sequential([
    Lambda(lambda x: x, input_shape=(66, 200, 3)),
    Conv2D(24, (5,5), strides=(2,2), activation='relu'),
    Conv2D(36, (5,5), strides=(2,2), activation='relu'),
    Conv2D(48, (5,5), strides=(2,2), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Entrenamiento
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
# Guardar modelo
model.save('model_circuito.h5')